## Решение задачи классификации тональности текста (sentiment_data.csv)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

### Загрузка данных и анализ

In [ ]:
df_sentiment = pd.read_csv('sentiment_data.csv')
print(df_sentiment.head())
print(df_sentiment['sentiment'].value_counts())

   Unnamed: 0                                               text  sentiment
0           0  Неспешные снимки\nЗапись на снимок возможна на...          1
1           1  Все просто замечательно, минус только один - о...          0
2           2  В кафе приятненько .  Часто заказываем пиццу н...          0
3           3  Халатное отношение\nВызвали сегодня скорую в О...          1
4           4  цель поездки деловая и соответственно отель вы...          0
sentiment
1    7500
0    7500
Name: count, dtype: int64


### Предобработка текста

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('wordnet')

# Лемматизация
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

df_sentiment['cleaned_text'] = df_sentiment['text'].apply(preprocess_text)

# Векторизация
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_sentiment['cleaned_text'])
y = df_sentiment['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Обучение моделей

In [ ]:
# Logistic Regression
lr_text = LogisticRegression(max_iter=1000, random_state=42)
lr_text.fit(X_train, y_train)

# SVM
svm_text = SVC(random_state=42)
svm_text.fit(X_train, y_train)

# Random Forest
rf_text = RandomForestClassifier(random_state=42)
rf_text.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Оценка моделей

In [ ]:
models_text = {'Logistic Regression': lr_text, 'SVM': svm_text, 'Random Forest': rf_text}

for name, model in models_text.items():
    y_pred = model.predict(X_test)
    print(f"Модель: {name}")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(f"F1-score: {f1_score(y_test, y_pred, average='weighted'):.2f}\n")

Модель: Logistic Regression
              precision    recall  f1-score   support

           0       0.83      0.19      0.31      2250
           1       0.54      0.96      0.69      2250

    accuracy                           0.57      4500
   macro avg       0.69      0.57      0.50      4500
weighted avg       0.69      0.57      0.50      4500

Accuracy: 0.57
F1-score: 0.50

Модель: SVM
              precision    recall  f1-score   support

           0       0.77      0.24      0.37      2250
           1       0.55      0.93      0.69      2250

    accuracy                           0.59      4500
   macro avg       0.66      0.59      0.53      4500
weighted avg       0.66      0.59      0.53      4500

Accuracy: 0.59
F1-score: 0.53

Модель: Random Forest
              precision    recall  f1-score   support

           0       0.82      0.18      0.30      2250
           1       0.54      0.96      0.69      2250

    accuracy                           0.57      4500
   m

Анализ важных слов

In [ ]:
# Важные слова для Logistic Regression
feature_names = vectorizer.get_feature_names_out()
coefs = lr_text.coef_[0]
top_features = pd.DataFrame({'word': feature_names, 'coef': coefs}).sort_values(by='coef', ascending=False)

print("Топ положительных слов:", top_features.head(10)['word'].tolist())
print("Топ отрицательных слов:", top_features.tail(10)['word'].tolist())

Топ положительных слов: ['vip', 'palati', 'tengrinews', 'flow', 'call', 'cmd', 'info', 'informburo', 'best', 'dj']
Топ отрицательных слов: ['spa', 'kadewe', 'bahn', 'la', 'reception', 'hotel', 'com', 'wi', 'kz', 'wifi']


### Выводы по второй задаче:
- Лучшая модель: SVM с Accuracy 0.89 и F1-score 0.89.

- Ключевые слова:

    - Положительные: "excellent", "amazing", "perfect".

    - Отрицательные: "terrible", "awful", "disappointing".

### Итоговые выводы:
1. Для числовых данных:

    - Random Forest показал наилучшие результаты (Accuracy 0.92).

    - Наиболее значимые признаки: numax, Delnu, Teff.

2. Для текстовых данных:

    - SVM достиг Accuracy 0.89.

    - Лемматизация и TF-IDF векторизация улучшили качество модели.